# Use this to record system audio to 'output.wav'
---

In [1]:
!pip install pyaudio

In [2]:
import pyaudio
import wave

In [3]:
chunk = 1024  # Number of frames in buffer
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1  # Stereo
fs = 44100  # Sample rate
seconds = 60  # Duration of recording

In [4]:
p = pyaudio.PyAudio()  # Create an interface to PortAudio

In [5]:
# Check available audio devices
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))


Input Device id  0  -  Microsoft Sound Mapper - Input
Input Device id  1  -  Stereo Mix (Realtek(R) Audio)
Input Device id  2  -  Microphone (DroidCam Virtual Au
Input Device id  3  -  Microphone Array (Realtek(R) Au


In [6]:
# Set the audio input device to Stereo Mix or What U Hear
device_index = None
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    if 'Stereo Mix' in dev['name']:
        device_index = dev['index']
        break
    elif 'What U Hear' in dev['name']:
        device_index = dev['index']
        break


In [7]:
# Open the audio stream
stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True,
                input_device_index=device_index)

frames = []  # Initialize array to store frames

# Record audio in chunks
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream
stream.stop_stream()
stream.close()

# Terminate the PortAudio interface
p.terminate()

# Save the recorded data as a WAV file
wf = wave.open("output.wav", "wb")
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b"".join(frames))
wf.close()
